In [1]:
import datanews
from datetime import date, timedelta, datetime
from dateutil.rrule import rrule, DAILY
from tqdm import tqdm
import pandas as pd
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
datanews.api_key = '0j4msa1c7uwnpv9v3air2bz66'

In [2]:
frames=[]
delta = timedelta(days=1)
start_date = date(2019, 1, 1)
end_date = date(2021, 6, 3)
for i in tqdm(range(880)):
    s = start_date.strftime("%Y-%m-%d")
    e = (start_date+delta).strftime("%Y-%m-%d")
    response = datanews.news(q='Doge',from_date=s,to_date=e)
    articles = response['hits'] 
    d = pd.DataFrame(articles)  
    frames.append(d)
    start_date+=delta
    if start_date>end_date:
      break
    time.sleep(1.5)

100%|██████████| 880/880 [25:57<00:00,  1.77s/it]


In [44]:
data = pd.concat(frames)
data.to_csv("new_daily.csv")

In [45]:
data.drop(columns=['url', 'source', 'authors','country', 'language',
       'description', 'imageUrl', 'content'],inplace=True)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [46]:
x = data['pubDate'].tolist()
for i in range(len(x)):
  x[i] = x[i].split('T')[0]
data['pubDate']=pd.DataFrame(x)

In [47]:
sent=[]
li = data['title'].tolist()
for i in tqdm(li):
  sent.append(sentiment_score(i))

100%|██████████| 2372/2372 [05:24<00:00,  7.31it/s]


In [56]:
ss={}

ss['Sent'] = sent
ss['date'] = x
s = pd.DataFrame(ss)

In [59]:
s.tail()

,Sent,date
2367,1,2021-05-29
2368,1,2021-05-30
2369,4,2021-05-29
2370,1,2021-05-30
2371,5,2021-05-30


In [80]:
sent_date=s.groupby(['date']).mean()

In [63]:
sent_date.to_csv("date_sent.csv")

In [81]:
sent_date.head()

,Sent
date,
2019-01-24,1.0
2019-04-01,5.0
2019-07-05,5.0
2019-08-30,1.0
2019-10-09,5.0
